**1. Self Attention📌**

아래 코드를 수행해보고, Self Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명:

 self attention은 모든 단어들이 전부 한 번씩 Query가 되는 것이라고 할 수 있다.

 attention의 결과는 attention value(실시간 context vector), 즉 Query의 정보가 반영된 key 정보의 가중치 합이다.

 모든 단어들이 전부 한 번씩 쿼리가 되어 context embedding을 함으로써, 각 단어의 의미를 그 자신을 포함한 모든 단어들의 영향을 바탕으로 결정하는 것이다. 즉, 그 단어 하나만의 사전적 의미가 아닌, 문장 내 맥락에서 가지는 의미도 포함하는 context vector를 시퀀스마다 실시간으로 만드는 것이다.

In [ ]:
import numpy as np

# 간단한 문장: ['나는', '사과를', '먹었다']
words = ['나는', '사과를', '먹었다']
word_vectors = {
    '나는': np.array([1, 0, 0]),
    '사과를': np.array([0, 1, 0]),
    '먹었다': np.array([0, 0, 1])
}

# 유사도 계산 함수 (self-attention)
def self_attention(query, key):
    return np.dot(query, key) # 쿼리와 키의 내적을 통해 유사도를 계산한다.

attention_matrix = np.zeros((len(words), len(words)))
for i in range(len(words)):
    for j in range(len(words)):
        attention_matrix[i][j] = self_attention(word_vectors[words[i]], word_vectors[words[j]])
        ''' words의 각 요소(토큰)이 쿼리이자 키로 쓰인다.
        이중 for문을 통해 모든 단어 쌍에 대해 한 번씩 내적하여,
        self attention(자기 자신을 포함한 모든 단어들의 영향을 반영)을 수행한다.'''

print("Self-Attention Matrix:")
print(attention_matrix)


Self-Attention Matrix:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


**2. Multi-Head Self Attention📌**

아래 코드를 수행해보고, Multi-Head Self Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명:

 Multi-Head Self Attention은 encoder 내 각 단어에 대해서 self attention을 동시에 여러 번 수행하는 것(중의성 해소를 위해)으로, 여러 개의 head가 병렬적으로 self attention을 수행한다.

In [ ]:
# 여러 개의 attention heads
def multi_head_self_attention(query, key, heads=3):
    return [np.dot(query, key) for _ in range(heads)] # head 개수는 3개가 기본값

multi_head_attention_matrix = np.zeros((len(words), len(words), 3))
for i in range(len(words)):
    for j in range(len(words)):
        multi_head_attention_matrix[i][j] = multi_head_self_attention(word_vectors[words[i]], word_vectors[words[j]])
        ''' multi_head_attention_matrix 함수에 포함된 for 문에 의해
        multi_head_attention_matrix[i][j]에는 3개의 어텐션 값이 저장된다.
        이 값은 i번째 단어와 j번째 단어 간의 attention 값으로,
        각 헤드별로 계산된 결과를 개별적으로 저장하고 있다.'''

print("\nMulti-Head Self-Attention Matrix:")
print(multi_head_attention_matrix)



Multi-Head Self-Attention Matrix:
[[[1. 1. 1.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [1. 1. 1.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [1. 1. 1.]]]


**3. Masked Multi-Head Self Attention📌**

아래 코드를 수행해보고, Masked Multi-Head Self Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명:

 t시점에서 decoder 내 self attention을 구할 때 t+1 시점부터의 정보는 활용하지 않는 것이다. 디코더는 실시간으로 번역 중인 상황이기 때문에 미래 시점의 정보는 참고할 수 없기 때문이다.

In [ ]:
# 마스크 추가: 현재 단어 이후의 단어는 계산하지 않음
def masked_attention(query, key, mask):
    return np.dot(query, key) * mask

mask = np.array([1, 1, 0])  # 첫 번째, 두 번째는 보지만, 세 번째는 ______ (이 빈칸 꼭 채워주세요 : 보지 않는다.) )
masked_attention_matrix = np.zeros((len(words), len(words)))
for i in range(len(words)):
    for j in range(len(words)):
        masked_attention_matrix[i][j] = masked_attention(word_vectors[words[i]], word_vectors[words[j]], mask[j])
        '''내적한 후(어텐션 값을 구한 후) mask와 요소곱을 수행하여 현재 단어 이후의 단어(3번째)는 보지 않는다.'''

print("\nMasked Self-Attention Matrix:")
print(masked_attention_matrix)



Masked Self-Attention Matrix:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]


**4. Cross Attention📌**

아래 코드를 수행해보고, Cross Attention은 어떤 메커니즘인지 설명하고,

 그 설명에 맞게 각 코드에 직접 주석을 달아봅시다.

 ✅설명:

 query는 디코더 쪽의 것, value와 key는 인코더 쪽의 것을 활용하여 attention을 수행하는 것이다. 즉, 인코더와 디코더를 교차하여 attention을 수행한다.

In [ ]:
# 입력 문장과 응답 문장
question_words = ['너는', '사과를']
answer_words = ['나는', '먹었다']
question_vectors = {
    '너는': np.array([1, 0]),
    '사과를': np.array([0, 1])
}
answer_vectors = {
    '나는': np.array([1, 0]),
    '먹었다': np.array([0, 1])
}

# Cross-Attention
cross_attention_matrix = np.zeros((len(question_words), len(answer_words)))
for i in range(len(question_words)):
    for j in range(len(answer_words)):
        cross_attention_matrix[i][j] = np.dot(question_vectors[question_words[i]], answer_vectors[answer_words[j]])
        '''cross_attention_matrix[i][j]는 question_words의 i번째 단어(인코더)와
        answer_words의 j번째 단어(디코더) 사이의 어텐션 값(내적 결과)을 저장한다.
        입력 문장과 응답 문장 간의 유사도를 계산하는 과정이다.'''

print("\nCross-Attention Matrix:")
print(cross_attention_matrix)


Cross-Attention Matrix:
[[1. 0.]
 [0. 1.]]
